In [ ]:
# This code is in reference to an interesting game relating to information theory and proability that I encountered and was stumped by.
# I try various different algorithms to try and develop a strategy with the highest expected profit.

# Here is the description of the game:
# In this game, you're presented with three coins, marked X, Y, and Z. Each coin has a different likelihood of landing on tails.
# The likelihoods are 0.7, 0.5, and 0.3 but you aren't informed which likelihood corresponds to which coin. 
# The game offers two methods to gather information, both involving a cost.
# You can use both these methods however much you wish:
    # ﻿﻿For a fee of $20, you can choose any two coins, flip them, and be informed about the total number of tails obtained in the pair. 
        #However, note that you won't know the specific outcome of each coin, only the combined total (i.e. you are told that either 0, 1, or 2 coins landed on tails from the pair)
    # ﻿﻿The second option incurs a fee of $60. This lets you pick a single coin, flip it, and observe which side it lands on.

# The objective is to try determine which coin has the highest probability of landing on tails. Successfully doing so will earn you $800. You want to
# choose a coin once you have a certain level of confidence that it is the correct coin, and when paying more to receive more information isn't worth it 
# in terms of expected value.

In [9]:



# This strategy finds the average profit of an initial conjecture of mine. 
# Note: this is not dynamic enough, so it serves as a base line.

import random

def simulate_game():
    # Randomly assign tail probabilities to coins X, Y, Z
    tail_probs = [0.7, 0.5, 0.3]
    random.shuffle(tail_probs)
    coins = {
        'X': tail_probs[0],
        'Y': tail_probs[1],
        'Z': tail_probs[2],
    }
    # Flip each coin individually once
    flip_results = {}
    for coin, prob in coins.items():
        flip = 'Tails' if random.random() < prob else 'Heads'
        flip_results[coin] = flip

    # Use Bayes' theorem to compute posterior probabilities
    prior_prob = 1/3  # Equal prior probability for each coin being the 0.7 coin
    posterior_probs = {}
    for coin in coins:
        # Likelihoods for each possible tail probability
        likelihoods = {
            0.7: 0.7 if flip_results[coin] == 'Tails' else 0.3,
            0.5: 0.5 if flip_results[coin] == 'Tails' else 0.5,
            0.3: 0.3 if flip_results[coin] == 'Tails' else 0.7,
        }
        # Numerator for Bayes' theorem
        numerator = likelihoods[0.7] * prior_prob
        # Denominator for Bayes' theorem
        denominator = sum(likelihood * prior_prob for likelihood in likelihoods.values())
        # Posterior probability for the coin being the 0.7 coin
        posterior_probs[coin] = numerator / denominator

    # Choose the coin with the highest posterior probability
    chosen_coin = max(posterior_probs, key=posterior_probs.get)
    # Check if the chosen coin is actually the 0.7 coin
    win = coins[chosen_coin] == 0.7
    # Calculate profit
    cost = 3 * 60  # $60 per individual coin flip
    profit = 800 - cost if win else -cost
    return profit

def strat1():
    total_games = 100000  # Number of simulations
    total_profit = 0
    for _ in range(total_games):
        total_profit += simulate_game()
    average_profit = total_profit / total_games
    print(f"Average profit over {total_games} games: ${average_profit:.2f}")

In [10]:
strat1()

Average profit over 100000 games: $213.78


In [11]:
# This strategy attempts to utilize a genetic algorithmic approach to solving it.

import random
import copy
import numpy as np

# Constants
NUM_COINS = 3
TAIL_PROBS = [0.7, 0.5, 0.3]
METHOD_A_COST = 20
METHOD_B_COST = 60
REWARD = 800

# Parameters for Genetic Algorithm
POPULATION_SIZE = 50
GENERATIONS = 50
MUTATION_RATE = 0.1
ELITE_SIZE = 5  # Number of top strategies to carry over to the next generation

# Strategy Representation
MAX_ACTIONS = 5  # Maximum number of actions in a strategy

class Strategy:
    def __init__(self):
        # Each action is a tuple: (action_type, coins)
        # action_type: 0 for Method A, 1 for Method B
        # coins: tuple of coin indices
        self.actions = [self.random_action() for _ in range(random.randint(1, MAX_ACTIONS))]
        self.max_cost = random.randint(60, 300)  # Maximum total cost allowed
        self.confidence_threshold = random.uniform(0.4, 0.9)  # Confidence threshold to stop

    def random_action(self):
        action_type = random.choice([0, 1])
        if action_type == 0:
            # Method A: Choose a pair of coins
            coins = tuple(sorted(random.sample(range(NUM_COINS), 2)))
        else:
            # Method B: Choose a single coin
            coins = (random.randint(0, NUM_COINS - 1),)
        return (action_type, coins)

    def mutate(self):
        # Randomly mutate the strategy
        if random.random() < 0.5 and self.actions:
            # Mutate an existing action
            idx = random.randint(0, len(self.actions) - 1)
            self.actions[idx] = self.random_action()
        else:
            # Add or remove an action
            if random.random() < 0.5 and len(self.actions) < MAX_ACTIONS:
                self.actions.append(self.random_action())
            elif self.actions:
                self.actions.pop(random.randint(0, len(self.actions) - 1))
        # Mutate max_cost and confidence_threshold
        self.max_cost = max(60, min(300, int(self.max_cost + random.gauss(0, 20))))
        self.confidence_threshold = min(0.9, max(0.4, self.confidence_threshold + random.uniform(-0.1, 0.1)))

    def crossover(self, other):
        # Crossover with another strategy
        child = Strategy()
        min_len = min(len(self.actions), len(other.actions))
        if min_len <= 1:
            # Cannot perform crossover; copy actions from one parent
            child.actions = self.actions.copy() if random.random() < 0.5 else other.actions.copy()
        else:
            split = random.randint(1, min_len - 1)
            child.actions = self.actions[:split] + other.actions[split:]
        # Ensure child has at least one action
        if not child.actions:
            child.actions = [self.random_action()]
        child.max_cost = random.choice([self.max_cost, other.max_cost])
        child.confidence_threshold = random.choice([self.confidence_threshold, other.confidence_threshold])
        return child

def simulate_strategy(strategy):
    total_profit = 0
    simulations = 1000  # Number of simulations to estimate expected profit
    for _ in range(simulations):
        profit = run_game(strategy)
        total_profit += profit
    average_profit = total_profit / simulations
    return average_profit

def run_game(strategy):
    # Randomly assign tail probabilities to coins
    tail_probs = TAIL_PROBS.copy()
    random.shuffle(tail_probs)
    coins = {i: tail_probs[i] for i in range(NUM_COINS)}
    prior_probs = {i: 1 / NUM_COINS for i in range(NUM_COINS)}
    total_cost = 0
    observations = []

    for action in strategy.actions:
        if total_cost >= strategy.max_cost:
            break  # Stop if max cost is reached

        action_type, selected_coins = action
        if action_type == 0:
            # Method A
            cost = METHOD_A_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            # Flip the selected coins and get the total number of tails
            tails_count = sum(1 for coin in selected_coins if random.random() < coins[coin])
            observations.append(('A', selected_coins, tails_count))
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_a(prior_probs, selected_coins, tails_count)
        else:
            # Method B
            cost = METHOD_B_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            coin = selected_coins[0]
            result = 'Tails' if random.random() < coins[coin] else 'Heads'
            observations.append(('B', coin, result))
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_b(prior_probs, coin, result)

        # Check confidence threshold
        most_probable_coin = max(prior_probs, key=prior_probs.get)
        if prior_probs[most_probable_coin] >= strategy.confidence_threshold:
            break

    # Make a guess: Choose the coin with the highest posterior probability of being the 0.7 coin
    guessed_coin = max(prior_probs, key=prior_probs.get)
    win = coins[guessed_coin] == 0.7
    profit = REWARD - total_cost if win else -total_cost
    return profit

def update_beliefs_method_a(prior_probs, selected_coins, tails_count):
    # Placeholder for belief update using Method A
    # Since detailed implementation is complex, we assume no change for this simulation
    return prior_probs

def update_beliefs_method_b(prior_probs, coin, result):
    # Update the prior probabilities based on the observation from Method B
    updated_probs = prior_probs.copy()
    # Possible tail probabilities and their likelihoods
    tail_probs = {0.7: 0, 0.5: 1, 0.3: 2}
    for c in prior_probs:
        # Update probability for this coin
        if c == coin:
            likelihoods = {
                0.7: 0.7 if result == 'Tails' else 0.3,
                0.5: 0.5,
                0.3: 0.3 if result == 'Tails' else 0.7,
            }
            # Calculate the likelihood of the observation
            likelihood = sum(likelihoods[prob] * (1 / NUM_COINS) for prob in likelihoods)
            updated_probs[c] *= likelihood
        else:
            # No change for other coins
            pass
    # Normalize probabilities
    total = sum(updated_probs.values())
    for c in updated_probs:
        updated_probs[c] /= total
    return updated_probs

def genetic_algorithm():
    # Initialize population
    population = [Strategy() for _ in range(POPULATION_SIZE)]
    for generation in range(GENERATIONS):
        print(f"Generation {generation + 1}")
        # Evaluate fitness
        fitness_results = []
        for idx, strategy in enumerate(population):
            fitness = simulate_strategy(strategy)
            fitness_results.append((fitness, strategy))
            # print(f"Strategy {idx + 1}: Expected Profit = ${fitness:.2f}")
        # Sort strategies by fitness
        fitness_results.sort(reverse=True, key=lambda x: x[0])
        # Keep the top strategies (elitism)
        new_population = [copy.deepcopy(fitness_results[i][1]) for i in range(ELITE_SIZE)]
        # Create new offspring
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(fitness_results)
            parent2 = tournament_selection(fitness_results)
            child = parent1.crossover(parent2)
            # Mutate child
            if random.random() < MUTATION_RATE:
                child.mutate()
            new_population.append(child)
        population = new_population
    # Return the best strategy
    best_strategy = fitness_results[0][1]
    best_fitness = fitness_results[0][0]
    print(f"\nBest Strategy Expected Profit: ${best_fitness:.2f}")
    return best_strategy

def tournament_selection(fitness_results):
    # Select a strategy using tournament selection
    tournament_size = min(5, len(fitness_results))
    competitors = random.sample(fitness_results, tournament_size)
    winner = max(competitors, key=lambda x: x[0])[1]
    return winner

def geneStrat():
    best_strategy = genetic_algorithm()
    # Display the best strategy
    print("\nBest Strategy:")
    for action in best_strategy.actions:
        action_type = 'Method A' if action[0] == 0 else 'Method B'
        coins = action[1]
        print(f"Action: {action_type}, Coins: {coins}")
    print(f"Maximum Cost Allowed: ${best_strategy.max_cost}")
    print(f"Confidence Threshold: {best_strategy.confidence_threshold:.2f}")


In [12]:
geneStrat()

Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30
Generation 31
Generation 32
Generation 33
Generation 34
Generation 35
Generation 36
Generation 37
Generation 38
Generation 39
Generation 40
Generation 41
Generation 42
Generation 43
Generation 44
Generation 45
Generation 46
Generation 47
Generation 48
Generation 49
Generation 50

Best Strategy Expected Profit: $272.00

Best Strategy:
Maximum Cost Allowed: $154
Confidence Threshold: 0.54


In [7]:
# This genetic algorithm also maps intermediate results, getting a tree like tracking of response to intermediate results.
# I believe the results of this algorithm are the most promising.

import random
import copy
import numpy as np

# Constants
NUM_COINS = 3
TAIL_PROBS = [0.7, 0.5, 0.3]
METHOD_A_COST = 20
METHOD_B_COST = 60
REWARD = 800

# Parameters for Genetic Algorithm
POPULATION_SIZE = 200
GENERATIONS = 500
MUTATION_RATE = 0.05
ELITE_SIZE = int(POPULATION_SIZE * 0.07)  # Number of top strategies to carry over to the next generation

# Strategy Representation
MAX_DEPTH = 10  # Maximum depth of the decision tree

# Possible Assignments (6 permutations)
from itertools import permutations

assignments = []
for perm in permutations(TAIL_PROBS):
    assignments.append({i: prob for i, prob in enumerate(perm)})

NUM_ASSIGNMENTS = len(assignments)

class Node:
    def __init__(self, action_type=None, coins=None):
        self.action_type = action_type  # 0 for Method A, 1 for Method B
        self.coins = coins  # tuple of coin indices
        self.children = {}  # mapping from observations to child nodes
        self.is_leaf = False  # If True, this node represents making a guess
        self.guess = None  # The coin to guess as the 0.7 coin (if is_leaf is True)

class Strategy:
    def __init__(self):
        self.root = self.random_subtree()
        self.max_cost = random.randint(60, 300)  # Maximum total cost allowed
        self.confidence_threshold = random.uniform(0.4, 0.99)  # Confidence threshold to stop

    def random_subtree(self, depth=0):
        # Create a random subtree
        if depth >= MAX_DEPTH or random.random() < 0.3:
            # Create a leaf node (make a guess)
            node = Node()
            node.is_leaf = True
            node.guess = random.randint(0, NUM_COINS - 1)
            return node
        else:
            # Create an action node
            action_type = random.choice([0, 1])
            if action_type == 0:
                # Method A: Choose a pair of coins
                coins = tuple(sorted(random.sample(range(NUM_COINS), 2)))
            else:
                # Method B: Choose a single coin
                coins = (random.randint(0, NUM_COINS - 1),)
            node = Node(action_type, coins)
            # For each possible observation, create child nodes
            observations = get_possible_observations(action_type)
            for obs in observations:
                node.children[obs] = self.random_subtree(depth + 1)
            return node

    def mutate(self):
        # Mutate the tree
        self.mutate_subtree(self.root, depth=0)

        # Mutate max_cost and confidence_threshold
        self.max_cost = max(60, min(300, int(self.max_cost + random.gauss(0, 20))))
        self.confidence_threshold = min(0.99, max(0.4, self.confidence_threshold + random.uniform(-0.05, 0.05)))

    def mutate_subtree(self, node, depth):
        if node.is_leaf:
            # Possibly change the guess
            if random.random() < 0.2:
                node.guess = random.randint(0, NUM_COINS - 1)
        else:
            # Possibly change the action
            if random.random() < 0.2:
                node.action_type = random.choice([0, 1])
                if node.action_type == 0:
                    # Method A: Choose a pair of coins
                    node.coins = tuple(sorted(random.sample(range(NUM_COINS), 2)))
                else:
                    # Method B: Choose a single coin
                    node.coins = (random.randint(0, NUM_COINS - 1),)
                # Update possible observations and children
                observations = get_possible_observations(node.action_type)
                # Remove any children for observations that no longer exist
                node.children = {obs: child for obs, child in node.children.items() if obs in observations}
                # Add new children for any new observations
                for obs in observations:
                    if obs not in node.children:
                        node.children[obs] = self.random_subtree(depth=depth + 1)
            # Recursively mutate children
            for child in node.children.values():
                self.mutate_subtree(child, depth=depth + 1)

    def crossover(self, other):
        child = Strategy()
        child.root = self.crossover_subtrees(self.root, other.root, depth=0)
        child.max_cost = random.choice([self.max_cost, other.max_cost])
        child.confidence_threshold = random.choice([self.confidence_threshold, other.confidence_threshold])
        return child

    def crossover_subtrees(self, node1, node2, depth):
        if depth >= MAX_DEPTH or (node1.is_leaf and node2.is_leaf):
            # Randomly choose one of the leaf nodes
            return copy.deepcopy(random.choice([node1, node2]))
        elif node1.is_leaf:
            return copy.deepcopy(node1)
        elif node2.is_leaf:
            return copy.deepcopy(node2)
        else:
            # Create a new node combining actions
            new_node = Node()
            new_node.action_type = random.choice([node1.action_type, node2.action_type])

            # Select coins appropriate for the action type
            coins_options = []
            if node1.action_type == new_node.action_type:
                coins_options.append(node1.coins)
            if node2.action_type == new_node.action_type:
                coins_options.append(node2.coins)

            if coins_options:
                new_node.coins = random.choice(coins_options)
            else:
                # No parent has matching action_type, generate new coins
                if new_node.action_type == 0:
                    new_node.coins = tuple(sorted(random.sample(range(NUM_COINS), 2)))
                else:
                    new_node.coins = (random.randint(0, NUM_COINS - 1),)

            # Combine children
            observations = get_possible_observations(new_node.action_type)
            new_node.children = {}
            for obs in observations:
                child1 = node1.children.get(obs)
                child2 = node2.children.get(obs)
                if child1 and child2:
                    new_node.children[obs] = self.crossover_subtrees(child1, child2, depth=depth + 1)
                elif child1:
                    new_node.children[obs] = copy.deepcopy(child1)
                elif child2:
                    new_node.children[obs] = copy.deepcopy(child2)
                else:
                    new_node.children[obs] = self.random_subtree(depth=depth + 1)
            return new_node

def get_possible_observations(action_type):
    if action_type == 0:
        # Method A: flipping two coins, possible tails counts are 0, 1, 2
        return [0, 1, 2]
    else:
        # Method B: flipping one coin, possible results are 'Heads', 'Tails'
        return ['Heads', 'Tails']

def simulate_strategy(strategy):
    total_profit = 0
    simulations = 1000  # Number of simulations to estimate expected profit
    for _ in range(simulations):
        profit = run_game(strategy)
        total_profit += profit
    average_profit = total_profit / simulations
    return average_profit

def run_game(strategy):
    true_assignment_index = random.randint(0, NUM_ASSIGNMENTS - 1)
    true_assignment = assignments[true_assignment_index]

    # Initialize prior probabilities over assignments
    prior_probs = {i: 1 / NUM_ASSIGNMENTS for i in range(NUM_ASSIGNMENTS)}

    total_cost = 0
    node = strategy.root

    while True:
        if total_cost >= strategy.max_cost or node.is_leaf:
            break  # Stop if max cost is reached or we are at a leaf node

        action_type = node.action_type
        selected_coins = node.coins

        if action_type == 0:
            # Method A
            cost = METHOD_A_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            # Flip the selected coins and get the total number of tails
            tails_count = sum(1 for coin in selected_coins if random.random() < true_assignment[coin])
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_a(prior_probs, selected_coins, tails_count)
            observation = tails_count
        else:
            # Method B
            cost = METHOD_B_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            coin = selected_coins[0]
            result = 'Tails' if random.random() < true_assignment[coin] else 'Heads'
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_b(prior_probs, coin, result)
            observation = result

        # Move to the next node based on the observation
        if observation in node.children:
            node = node.children[observation]
        else:
            # Observation not anticipated in the tree, break
            break

    # Make a guess: Choose the coin with the highest posterior probability of being the 0.7 coin
    if node.is_leaf and node.guess is not None:
        guessed_coin = node.guess
    else:
        coin_probs = compute_coin_probs(prior_probs)
        guessed_coin = max(coin_probs, key=coin_probs.get)

    win = true_assignment[guessed_coin] == 0.7
    profit = REWARD - total_cost if win else -total_cost
    return profit

def update_beliefs_method_a(prior_probs, selected_coins, tails_count):
    # Update the prior probabilities over assignments based on the observation from Method A
    updated_probs = {}
    total_prob = 0
    for idx, assignment in enumerate(assignments):
        # Calculate the likelihood of getting tails_count tails from selected_coins under this assignment
        probs = [assignment[coin] for coin in selected_coins]
        likelihood = binomial_likelihood(probs, tails_count)
        updated_prob = likelihood * prior_probs[idx]
        updated_probs[idx] = updated_prob
        total_prob += updated_prob
    # Normalize probabilities
    for idx in updated_probs:
        if total_prob > 0:
            updated_probs[idx] /= total_prob
        else:
            # Avoid division by zero
            updated_probs[idx] = 0
    return updated_probs

def binomial_likelihood(probs, tails_count):
    # Calculate the probability of getting tails_count tails from coins with given probabilities
    if len(probs) == 2:
        if tails_count == 0:
            # Both coins are heads
            return (1 - probs[0]) * (1 - probs[1])
        elif tails_count == 1:
            # One coin is tails, one is heads
            return probs[0] * (1 - probs[1]) + (1 - probs[0]) * probs[1]
        elif tails_count == 2:
            # Both coins are tails
            return probs[0] * probs[1]
        else:
            return 0  # Should not happen
    elif len(probs) == 1:
        # Should not happen in Method A, but handle just in case
        if tails_count == 0:
            return 1 - probs[0]
        elif tails_count == 1:
            return probs[0]
        else:
            return 0
    else:
        return 0

def update_beliefs_method_b(prior_probs, coin, result):
    # Update the prior probabilities over assignments based on the observation from Method B
    updated_probs = {}
    total_prob = 0
    for idx, assignment in enumerate(assignments):
        coin_prob = assignment[coin]
        likelihood = coin_prob if result == 'Tails' else (1 - coin_prob)
        updated_prob = likelihood * prior_probs[idx]
        updated_probs[idx] = updated_prob
        total_prob += updated_prob
    # Normalize probabilities
    for idx in updated_probs:
        if total_prob > 0:
            updated_probs[idx] /= total_prob
        else:
            updated_probs[idx] = 0
    return updated_probs

def compute_coin_probs(prior_probs):
    # Compute the posterior probabilities for each coin being the 0.7 coin
    coin_probs = {i: 0 for i in range(NUM_COINS)}
    for idx, prob in prior_probs.items():
        assignment = assignments[idx]
        for coin in range(NUM_COINS):
            if assignment[coin] == 0.7:
                coin_probs[coin] += prob
    return coin_probs

def genetic_algorithm():
    # Initialize population
    population = [Strategy() for _ in range(POPULATION_SIZE)]
    for generation in range(GENERATIONS):
        print(f"Generation {generation + 1}")
        # Evaluate fitness
        fitness_results = []
        for idx, strategy in enumerate(population):
            fitness = simulate_strategy(strategy)
            fitness_results.append((fitness, strategy))
            # Optionally print progress
            # print(f"Strategy {idx + 1}: Expected Profit = ${fitness:.2f}")
        # Sort strategies by fitness
        fitness_results.sort(reverse=True, key=lambda x: x[0])
        # Keep the top strategies (elitism)
        new_population = [copy.deepcopy(fitness_results[i][1]) for i in range(ELITE_SIZE)]
        # Create new offspring
        while len(new_population) < POPULATION_SIZE:
            parent1 = tournament_selection(fitness_results)
            parent2 = tournament_selection(fitness_results)
            child = parent1.crossover(parent2)
            # Mutate child
            if random.random() < MUTATION_RATE:
                child.mutate()
            new_population.append(child)
        population = new_population
    # Return the best strategy
    best_strategy = fitness_results[0][1]
    best_fitness = fitness_results[0][0]
    print(f"\nBest Strategy Expected Profit: ${best_fitness:.2f}")
    return best_strategy

def tournament_selection(fitness_results):
    # Select a strategy using tournament selection
    tournament_size = min(5, len(fitness_results))
    competitors = random.sample(fitness_results, tournament_size)
    winner = max(competitors, key=lambda x: x[0])[1]
    return winner

def print_strategy(node, indent=""):
    if node.is_leaf:
        print(indent + f"Guess Coin: {node.guess}")
    else:
        action_type = 'Method A' if node.action_type == 0 else 'Method B'
        print(indent + f"Action: {action_type}, Coins: {node.coins}")
        for obs, child in node.children.items():
            print(indent + f"  If {obs}:")
            print_strategy(child, indent + "    ")

def geneStrat2():
    best_strategy = genetic_algorithm()
    # Display the best strategy
    print("\nBest Strategy:")
    print_strategy(best_strategy.root)
    print(f"Maximum Cost Allowed: ${best_strategy.max_cost}")
    print(f"Confidence Threshold: {best_strategy.confidence_threshold:.2f}")

# Run the genetic algorithm to find the best strategy
if __name__ == "__main__":
    geneStrat2()


Generation 1
Generation 2
Generation 3
Generation 4
Generation 5
Generation 6
Generation 7
Generation 8
Generation 9
Generation 10
Generation 11
Generation 12
Generation 13
Generation 14
Generation 15
Generation 16
Generation 17
Generation 18
Generation 19
Generation 20
Generation 21
Generation 22
Generation 23
Generation 24
Generation 25
Generation 26
Generation 27
Generation 28
Generation 29
Generation 30
Generation 31
Generation 32
Generation 33
Generation 34
Generation 35
Generation 36
Generation 37
Generation 38
Generation 39
Generation 40
Generation 41
Generation 42
Generation 43
Generation 44
Generation 45
Generation 46
Generation 47
Generation 48
Generation 49
Generation 50
Generation 51
Generation 52
Generation 53
Generation 54
Generation 55
Generation 56
Generation 57
Generation 58
Generation 59
Generation 60
Generation 61
Generation 62
Generation 63
Generation 64
Generation 65
Generation 66
Generation 67
Generation 68
Generation 69
Generation 70
Generation 71
Generation 72
G

In [22]:
# Genetic algorithm but with variable hyperparameters that are tested to see which is the best

import random
import copy
import numpy as np
from itertools import permutations

# Constants
NUM_COINS = 3
TAIL_PROBS = [0.7, 0.5, 0.3]
METHOD_A_COST = 20
METHOD_B_COST = 60
REWARD = 800

# Possible Assignments (6 permutations)
assignments = []
for perm in permutations(TAIL_PROBS):
    assignments.append({i: prob for i, prob in enumerate(perm)})

NUM_ASSIGNMENTS = len(assignments)

# Strategy Representation
MAX_ACTIONS = 5  # Maximum number of actions in a strategy

class Strategy:
    def __init__(self):
        # Each action is a tuple: (action_type, coins)
        # action_type: 0 for Method A, 1 for Method B
        # coins: tuple of coin indices
        self.actions = [self.random_action() for _ in range(random.randint(1, MAX_ACTIONS))]
        self.max_cost = random.randint(60, 300)  # Maximum total cost allowed
        self.confidence_threshold = random.uniform(0.4, 0.99)  # Confidence threshold to stop

    def random_action(self):
        action_type = random.choice([0, 1])
        if action_type == 0:
            # Method A: Choose a pair of coins
            coins = tuple(sorted(random.sample(range(NUM_COINS), 2)))
        else:
            # Method B: Choose a single coin
            coins = (random.randint(0, NUM_COINS - 1),)
        return (action_type, coins)

    def mutate(self):
        # Randomly mutate the strategy
        if random.random() < 0.5 and self.actions:
            # Mutate an existing action
            idx = random.randint(0, len(self.actions) - 1)
            self.actions[idx] = self.random_action()
        else:
            # Add or remove an action
            if random.random() < 0.5 and len(self.actions) < MAX_ACTIONS:
                self.actions.append(self.random_action())
            elif self.actions:
                self.actions.pop(random.randint(0, len(self.actions) - 1))
        # Mutate max_cost and confidence_threshold
        self.max_cost = max(60, min(300, int(self.max_cost + random.gauss(0, 20))))
        self.confidence_threshold = min(0.99, max(0.4, self.confidence_threshold + random.uniform(-0.05, 0.05)))

    def crossover(self, other):
        # Crossover with another strategy
        child = Strategy()
        min_len = min(len(self.actions), len(other.actions))
        if min_len <= 1:
            # Cannot perform crossover; copy actions from one parent
            child.actions = self.actions.copy() if random.random() < 0.5 else other.actions.copy()
        else:
            split = random.randint(1, min_len - 1)
            child.actions = self.actions[:split] + other.actions[split:]
        # Ensure child has at least one action
        if not child.actions:
            child.actions = [self.random_action()]
        child.max_cost = random.choice([self.max_cost, other.max_cost])
        child.confidence_threshold = random.choice([self.confidence_threshold, other.confidence_threshold])
        return child

def simulate_strategy(strategy):
    total_profit = 0
    simulations = 500  # Reduced number of simulations to limit runtime
    for _ in range(simulations):
        profit = run_game(strategy)
        total_profit += profit
    average_profit = total_profit / simulations
    return average_profit

def run_game(strategy):
    # Randomly select one of the possible assignments as the true assignment
    true_assignment_index = random.randint(0, NUM_ASSIGNMENTS - 1)
    true_assignment = assignments[true_assignment_index]

    # Initialize prior probabilities over assignments
    prior_probs = {i: 1 / NUM_ASSIGNMENTS for i in range(NUM_ASSIGNMENTS)}

    total_cost = 0

    # Initialize coin_probs before the loop
    coin_probs = compute_coin_probs(prior_probs)

    for action in strategy.actions:
        if total_cost >= strategy.max_cost:
            break  # Stop if max cost is reached

        action_type, selected_coins = action
        if action_type == 0:
            # Method A
            cost = METHOD_A_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            # Flip the selected coins and get the total number of tails
            tails_count = sum(1 for coin in selected_coins if random.random() < true_assignment[coin])
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_a(prior_probs, selected_coins, tails_count)
        else:
            # Method B
            cost = METHOD_B_COST
            if total_cost + cost > strategy.max_cost:
                break
            total_cost += cost
            coin = selected_coins[0]
            result = 'Tails' if random.random() < true_assignment[coin] else 'Heads'
            # Update beliefs based on observation
            prior_probs = update_beliefs_method_b(prior_probs, coin, result)

        # Compute posterior probabilities for each coin being the 0.7 coin
        coin_probs = compute_coin_probs(prior_probs)
        # Check confidence threshold
        most_probable_coin = max(coin_probs, key=coin_probs.get)
        if coin_probs[most_probable_coin] >= strategy.confidence_threshold:
            break

    # Make a guess: Choose the coin with the highest posterior probability of being the 0.7 coin
    guessed_coin = max(coin_probs, key=coin_probs.get)
    win = true_assignment[guessed_coin] == 0.7
    profit = REWARD - total_cost if win else -total_cost
    return profit

def update_beliefs_method_a(prior_probs, selected_coins, tails_count):
    # Update the prior probabilities over assignments based on the observation from Method A
    updated_probs = {}
    total_prob = 0
    for idx, assignment in enumerate(assignments):
        # Calculate the likelihood of getting tails_count tails from selected_coins under this assignment
        probs = [assignment[coin] for coin in selected_coins]
        likelihood = binomial_likelihood(probs, tails_count)
        updated_prob = likelihood * prior_probs[idx]
        updated_probs[idx] = updated_prob
        total_prob += updated_prob
    # Normalize probabilities
    for idx in updated_probs:
        updated_probs[idx] /= total_prob
    return updated_probs

def binomial_likelihood(probs, tails_count):
    # Calculate the probability of getting tails_count tails from coins with given probabilities
    # For two coins, calculate directly
    if tails_count == 0:
        # Both coins are heads
        return (1 - probs[0]) * (1 - probs[1])
    elif tails_count == 1:
        # One coin is tails, one is heads
        return probs[0] * (1 - probs[1]) + (1 - probs[0]) * probs[1]
    elif tails_count == 2:
        # Both coins are tails
        return probs[0] * probs[1]
    else:
        return 0  # Should not happen

def update_beliefs_method_b(prior_probs, coin, result):
    # Update the prior probabilities over assignments based on the observation from Method B
    updated_probs = {}
    total_prob = 0
    for idx, assignment in enumerate(assignments):
        coin_prob = assignment[coin]
        likelihood = coin_prob if result == 'Tails' else (1 - coin_prob)
        updated_prob = likelihood * prior_probs[idx]
        updated_probs[idx] = updated_prob
        total_prob += updated_prob
    # Normalize probabilities
    for idx in updated_probs:
        updated_probs[idx] /= total_prob
    return updated_probs

def compute_coin_probs(prior_probs):
    # Compute the posterior probabilities for each coin being the 0.7 coin
    coin_probs = {i: 0 for i in range(NUM_COINS)}
    for idx, prob in prior_probs.items():
        assignment = assignments[idx]
        for coin in range(NUM_COINS):
            if assignment[coin] == 0.7:
                coin_probs[coin] += prob
    return coin_probs

def genetic_algorithm(pop_size, mutation_rate, generations):
    # Initialize population
    population = [Strategy() for _ in range(pop_size)]
    for generation in range(generations):
        # Evaluate fitness
        fitness_results = []
        for strategy in population:
            fitness = simulate_strategy(strategy)
            fitness_results.append((fitness, strategy))
        # Sort strategies by fitness
        fitness_results.sort(reverse=True, key=lambda x: x[0])
        # Keep the top strategies (elitism)
        elite_size = max(1, int(0.1 * pop_size))  # Top 10% as elites
        new_population = [copy.deepcopy(fitness_results[i][1]) for i in range(elite_size)]
        # Create new offspring
        while len(new_population) < pop_size:
            parent1 = tournament_selection(fitness_results)
            parent2 = tournament_selection(fitness_results)
            child = parent1.crossover(parent2)
            # Mutate child
            if random.random() < mutation_rate:
                child.mutate()
            new_population.append(child)
        population = new_population
    # Return the best strategy
    best_strategy = fitness_results[0][1]
    best_fitness = fitness_results[0][0]
    return best_strategy, best_fitness

def tournament_selection(fitness_results):
    # Select a strategy using tournament selection
    tournament_size = min(5, len(fitness_results))
    competitors = random.sample(fitness_results, tournament_size)
    winner = max(competitors, key=lambda x: x[0])[1]
    return winner

def geneStrat2():
    # Hyperparameter ranges
    population_sizes = [30, 50, 70, 90]
    mutation_rates = [0.05, 0.1, 0.15, 0.2]
    generations_list = [10, 20, 30]
    
    best_overall_strategy = None
    best_overall_fitness = float('-inf')
    best_hyperparams = {}

    for pop_size in population_sizes:
        for mutation_rate in mutation_rates:
            for generations in generations_list:
                print(f"Testing Hyperparameters: Population Size={pop_size}, Mutation Rate={mutation_rate}, Generations={generations}")
                best_strategy, best_fitness = genetic_algorithm(pop_size, mutation_rate, generations)
                print(f"Best Strategy Expected Profit: ${best_fitness:.2f}\n")
                if best_fitness > best_overall_fitness:
                    best_overall_fitness = best_fitness
                    best_overall_strategy = best_strategy
                    best_hyperparams = {
                        'Population Size': pop_size,
                        'Mutation Rate': mutation_rate,
                        'Generations': generations
                    }

    # Display the best hyperparameters and strategy
    print("Best Hyperparameters Found:")
    for param, value in best_hyperparams.items():
        print(f"{param}: {value}")
    print(f"Best Expected Profit: ${best_overall_fitness:.2f}\n")

    print("Best Strategy:")
    for action in best_overall_strategy.actions:
        action_type = 'Method A' if action[0] == 0 else 'Method B'
        coins = action[1]
        print(f"Action: {action_type}, Coins: {coins}")
    print(f"Maximum Cost Allowed: ${best_overall_strategy.max_cost}")
    print(f"Confidence Threshold: {best_overall_strategy.confidence_threshold:.2f}")

Testing Hyperparameters: Population Size=30, Mutation Rate=0.05, Generations=10
Best Strategy Expected Profit: $356.08

Testing Hyperparameters: Population Size=30, Mutation Rate=0.05, Generations=20
Best Strategy Expected Profit: $347.20

Testing Hyperparameters: Population Size=30, Mutation Rate=0.05, Generations=30
Best Strategy Expected Profit: $343.28

Testing Hyperparameters: Population Size=30, Mutation Rate=0.1, Generations=10
Best Strategy Expected Profit: $343.92

Testing Hyperparameters: Population Size=30, Mutation Rate=0.1, Generations=20
Best Strategy Expected Profit: $355.20

Testing Hyperparameters: Population Size=30, Mutation Rate=0.1, Generations=30
Best Strategy Expected Profit: $366.16

Testing Hyperparameters: Population Size=30, Mutation Rate=0.15, Generations=10
Best Strategy Expected Profit: $340.80

Testing Hyperparameters: Population Size=30, Mutation Rate=0.15, Generations=20
Best Strategy Expected Profit: $357.60

Testing Hyperparameters: Population Size=30

In [2]:
# This code tries to use a brute force method but is limited in what can be done.
# It does not succeed as there are far too many possiblities. 


import itertools
from collections import defaultdict

# Define the coin names and possible probabilities
coins = ['X', 'Y', 'Z']
probabilities = [0.3, 0.5, 0.7]

# All possible assignments of probabilities to coins (6 permutations)
possible_permutations = list(itertools.permutations(probabilities))

def update_beliefs(beliefs, action, observation):
    """
    Update the belief state based on the action taken and the observation received.
    """
    new_beliefs = defaultdict(float)
    total_prob = 0.0

    for permutation, perm_prob in beliefs.items():
        # Calculate the probability of the observation given the permutation
        if action['type'] == 'flip_two':
            coin1, coin2 = action['coins']
            p1 = permutation[coins.index(coin1)]
            p2 = permutation[coins.index(coin2)]
            # Possible outcomes: 0, 1, or 2 tails
            if observation == 0:
                obs_prob = (1 - p1) * (1 - p2)
            elif observation == 1:
                obs_prob = p1 * (1 - p2) + (1 - p1) * p2
            elif observation == 2:
                obs_prob = p1 * p2
        elif action['type'] == 'flip_one':
            coin = action['coin']
            p = permutation[coins.index(coin)]
            obs_prob = p if observation == 'T' else (1 - p)
        else:
            obs_prob = 1  # No observation

        # Update the belief for this permutation
        new_beliefs[permutation] += perm_prob * obs_prob
        total_prob += perm_prob * obs_prob

    # Normalize the beliefs
    for perm in new_beliefs:
        new_beliefs[perm] /= total_prob

    return new_beliefs

def explore_paths(beliefs, total_cost, path, all_paths, max_total_cost, depth=0, path_counter=None):
    """
    Recursively explore all possible paths (every possible sequence of actions and observations).
    """
    if path_counter is None:
        path_counter = {'count': 0}

    # Compute the expected gain from making a guess now
    max_prob = 0
    best_coin = None
    coin_probs = {coin: 0.0 for coin in coins}

    for permutation, perm_prob in beliefs.items():
        for coin in coins:
            if permutation[coins.index(coin)] == 0.7:
                coin_probs[coin] += perm_prob

    for coin, prob in coin_probs.items():
        if prob > max_prob:
            max_prob = prob
            best_coin = coin

    # Record the current path with a guess
    guess_path = path + [('guess', best_coin)]
    all_paths.append({'path': guess_path, 'total_cost': total_cost, 'beliefs': beliefs})

    # Increment path counter
    path_counter['count'] += 1
    if path_counter['count'] % 10000 == 0:
        print(f"Number of paths explored so far: {path_counter['count']}")

    # If total cost exceeds maximum allowed cost, stop exploring further
    if total_cost >= max_total_cost:
        return

    # Explore all possible actions

    # Action 1: Flip two coins
    for coin_pair in itertools.combinations(coins, 2):
        action = {'type': 'flip_two', 'coins': coin_pair}
        cost = 20
        new_total_cost = total_cost + cost

        # Skip if new total cost exceeds maximum allowed cost
        if new_total_cost > max_total_cost:
            continue

        action_path = path + [("flip_two", coin_pair)]

        # Possible observations: 0, 1, or 2 tails
        for tails in [0, 1, 2]:
            observation_path = action_path + [("observe", tails)]
            new_beliefs = update_beliefs(beliefs, action, tails)
            explore_paths(new_beliefs, new_total_cost, observation_path, all_paths, max_total_cost, depth+1, path_counter)

    # Action 2: Flip one coin
    for coin in coins:
        action = {'type': 'flip_one', 'coin': coin}
        cost = 60
        new_total_cost = total_cost + cost

        # Skip if new total cost exceeds maximum allowed cost
        if new_total_cost > max_total_cost:
            continue

        action_path = path + [("flip_one", coin)]

        # Possible observations: 'H' or 'T'
        for outcome in ['H', 'T']:
            observation_path = action_path + [("observe", outcome)]
            new_beliefs = update_beliefs(beliefs, action, outcome)
            explore_paths(new_beliefs, new_total_cost, observation_path, all_paths, max_total_cost, depth+1, path_counter)

def evaluate_strategy(path, initial_beliefs):
    """
    Evaluate the expected profit of a given strategy (path).
    """
    beliefs = initial_beliefs.copy()
    total_cost = 0
    last_action = None

    for step in path:
        if step[0] == 'flip_two':
            last_action = {'type': 'flip_two', 'coins': step[1]}
            total_cost += 20
        elif step[0] == 'flip_one':
            last_action = {'type': 'flip_one', 'coin': step[1]}
            total_cost += 60
        elif step[0] == 'observe':
            observation = step[1]
            beliefs = update_beliefs(beliefs, last_action, observation)
            last_action = None
        elif step[0] == 'guess':
            # Compute expected profit based on current beliefs
            coin_probs = {coin: 0.0 for coin in coins}
            for permutation, perm_prob in beliefs.items():
                for coin in coins:
                    if permutation[coins.index(coin)] == 0.7:
                        coin_probs[coin] += perm_prob

            # Probability that the guessed coin is correct
            prob_correct = coin_probs[step[1]]

            expected_profit = prob_correct * 800 - total_cost
            return expected_profit, total_cost, prob_correct

    return None  # Should not reach here

def main():
    # Initial beliefs: uniform distribution over all permutations
    initial_beliefs = {perm: 1 / len(possible_permutations) for perm in possible_permutations}
    total_cost = 0
    path = []
    all_paths = []

    # Set the maximum total cost allowed
    max_total_cost = 100  # Adjust this value as needed

    path_counter = {'count': 0}

    explore_paths(initial_beliefs, total_cost, path, all_paths, max_total_cost, depth=0, path_counter=path_counter)

    # Evaluate each strategy
    strategies = []
    for path_info in all_paths:
        path = path_info['path']
        expected_profit, total_cost, prob_correct = evaluate_strategy(path, initial_beliefs)
        strategies.append({
            'path': path,
            'expected_profit': expected_profit,
            'total_cost': total_cost,
            'prob_correct': prob_correct
        })

    # Sort strategies by expected profit in descending order
    strategies.sort(key=lambda x: x['expected_profit'], reverse=True)

    # Print the top strategies
    print(f"Total number of strategies evaluated: {len(strategies)}\n")
    print("Top strategies ranked by expected profit:\n")

    for idx, strategy in enumerate(strategies[:5]):  # Adjust the number as needed
        print(f"Rank {idx + 1}: Expected Profit = ${strategy['expected_profit']:.2f}")
        print(f"Total Cost Incurred: ${strategy['total_cost']}")
        print(f"Probability of Correct Guess: {strategy['prob_correct'] * 100:.2f}%")
        print("Strategy Path:")
        for step in strategy['path']:
            if step[0] == 'guess':
                print(f"  Guess coin {step[1]}")
            elif step[0] == 'flip_two':
                print(f"  Flip two coins: {step[1]}")
            elif step[0] == 'flip_one':
                print(f"  Flip one coin: {step[1]}")
            elif step[0] == 'observe':
                print(f"    Observation: {step[1]}")
        print("-" * 40)

if __name__ == "__main__":
    main()


Number of paths explored so far: 10000
Number of paths explored so far: 20000
Number of paths explored so far: 30000
Number of paths explored so far: 40000
Number of paths explored so far: 50000
Number of paths explored so far: 60000
Total number of strategies evaluated: 68002

Top strategies ranked by expected profit:

Rank 1: Expected Profit = $632.45
Total Cost Incurred: $100
Probability of Correct Guess: 91.56%
Strategy Path:
  Flip two coins: ('X', 'Y')
    Observation: 0
  Flip two coins: ('X', 'Y')
    Observation: 0
  Flip two coins: ('X', 'Y')
    Observation: 0
  Flip two coins: ('X', 'Y')
    Observation: 0
  Flip two coins: ('X', 'Y')
    Observation: 0
  Guess coin Z
----------------------------------------
Rank 2: Expected Profit = $632.45
Total Cost Incurred: $100
Probability of Correct Guess: 91.56%
Strategy Path:
  Flip two coins: ('X', 'Z')
    Observation: 0
  Flip two coins: ('X', 'Z')
    Observation: 0
  Flip two coins: ('X', 'Z')
    Observation: 0
  Flip two coi